## Librerias Necesarias

In [ ]:
!pip install -U gdown

In [ ]:
import numpy as np
import pandas as pd
import random
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
import time
import ipywidgets as widgets
from IPython.display import display, clear_output
import os
import gdown
import zipfile
import matplotlib.pyplot as plt
import networkx as nx

## Carga de datos (nodos y matriz de distancias)

In [ ]:
if not os.path.exists("carpeta_descomprimida"):

  file_path = "https://drive.google.com/uc?id=1WW7AnTSgSVX7Ktn2z6GyHkQN3nbmqaRa"
  output = "archive.zip"
  gdown.download(file_path, output, quiet=False)


  with zipfile.ZipFile(output, 'r') as zip_ref:
      zip_ref.extractall('carpeta_descomprimida')

  os.remove(output)

Downloading...
From: https://drive.google.com/uc?id=1WW7AnTSgSVX7Ktn2z6GyHkQN3nbmqaRa
To: /content/archive.zip
100%|██████████| 847k/847k [00:00<00:00, 71.1MB/s]


## Clase Grafo

In [ ]:
class Graph:
    def __init__(self, name, folder):
        # Constructor de la clase Graph. Recibe el nombre del grafo y la carpeta donde están los archivos.
        self.name = name
        self.folder = folder

        # Rutas a los archivos parquet de nodos y de matriz de distancias.
        nodes_path = os.path.join(folder, f"{name}_nodes.parquet")
        dm_path = os.path.join(folder, f"{name}_dm.parquet")

        try:
            # Carga los datos de nodos desde el archivo Parquet.
            self.nodes_df = pd.read_parquet(nodes_path)
        except Exception as e:
            # Si ocurre un error al cargar, se lanza una excepción con un mensaje informativo.
            raise ValueError(f"Error cargando el archivo de nodos: {nodes_path}") from e

        try:
            # Carga la matriz de distancias como un arreglo NumPy desde archivo Parquet.
            self.dm = pd.read_parquet(dm_path).values
        except Exception as e:
            # Error personalizado si falla la carga de la matriz de distancias.
            raise ValueError(f"Error cargando el archivo de matriz de distancias: {dm_path}") from e

        # Número total de nodos en el grafo.
        self.num_nodes = len(self.nodes_df)

        # Número de aristas en un grafo completo no dirigido sin bucles (fórmula combinatoria).
        self.num_edges = (self.num_nodes * (self.num_nodes - 1)) // 2

    def __str__(self):
        # Representación en forma de cadena del objeto, útil para imprimir información resumida del grafo.
        return f"<Graph '{self.name}' with {self.num_nodes} nodes and {self.num_edges} edges>"

    def show_nodes(self):
        # Muestra por consola los nodos del grafo con sus coordenadas y demanda.
        print(f"Nodos del grafo '{self.name}':")
        for i, row in self.nodes_df.iterrows():
            print(f"  Nodo {i}: (x={row['x']}, y={row['y']}, demanda={row['demand']})")

    def get_neighbors(self, node_index):
        # Devuelve una lista de los índices de los nodos vecinos (todos los nodos excepto el actual).
        if not 0 <= node_index < self.num_nodes:
            raise ValueError(f"Índice inválido. Debe estar entre 0 y {self.num_nodes - 1}")
        return [i for i in range(self.num_nodes) if i != node_index]

    def show_distance_matrix(self):
        # Imprime la matriz de distancias como un DataFrame legible.
        print("Matriz de distancias:")
        print(pd.DataFrame(self.dm))

    def show_node_table(self):
        # Muestra la tabla de nodos completa.
        print("Tabla de Nodos:")
        print(self.nodes_df)

    def get_node_table(self):
        """Retorna una copia del DataFrame con los nodos para evitar modificaciones externas."""
        return self.nodes_df.copy()

    def get_distance_matrix(self):
        """Retorna la matriz de distancias como un DataFrame."""
        return pd.DataFrame(self.dm)


In [ ]:
folder = "carpeta_descomprimida/problemset/in"
nombre = "big-250n-c150_300-d15"

g = Graph(nombre, folder)
print(g)

<Graph 'big-250n-c150_300-d15' with 250 nodes and 31125 edges>


In [ ]:
g.get_node_table()  # Muestra tabla de nodos

,x,y,demand
0,92,77,0
1,29,47,15
2,48,99,15
3,31,59,15
4,45,68,15
...,...,...,...
245,50,87,15
246,31,46,15
247,26,1,15
248,23,38,15


In [ ]:
g.get_distance_matrix()  # Muestra la matriz de distancias

,0,1,2,3,4,5,6,7,8,9,...,240,241,242,243,244,245,246,247,248,249
0,0.000000,69.778220,49.193496,63.600314,47.853944,38.587563,27.018512,24.041631,85.070559,51.662365,...,78.390050,70.228199,59.665736,32.802439,21.931712,43.174066,68.425142,100.657836,79.259069,73.756356
1,69.778220,0.000000,55.362442,12.165525,26.400758,31.622777,68.680419,87.800911,18.439089,53.851648,...,44.944410,6.708204,18.788294,38.327536,65.069194,45.177428,2.236068,46.097722,10.816654,63.031738
2,49.193496,55.362442,0.000000,43.462628,31.144823,43.416587,27.018512,51.009803,60.605280,78.390050,...,90.138782,50.990195,60.033324,45.694639,28.861739,12.165525,55.659680,100.439036,65.924199,98.081599
3,63.600314,12.165525,43.462628,0.000000,16.643317,28.071338,58.796258,79.404030,21.540659,57.688820,...,54.626001,7.810250,25.000000,35.057096,55.803226,33.837849,13.000000,58.215118,22.472205,70.292247
4,47.853944,26.400758,31.144823,16.643317,0.000000,17.804494,42.426407,62.769419,38.013156,53.263496,...,59.539903,24.207437,29.017236,23.706539,39.204592,19.646883,26.076810,69.641941,37.202150,70.213959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,43.174066,45.177428,12.165525,33.837849,19.646883,31.320920,27.313001,50.695167,53.075418,66.640828,...,78.057671,41.617304,48.166378,34.000000,26.476405,0.000000,45.188494,89.286057,55.946403,86.023253
246,68.425142,2.236068,55.659680,13.000000,26.076810,30.083218,68.117545,86.833173,20.615528,51.623638,...,43.046487,8.602325,16.552945,36.687873,64.350602,45.188494,0.000000,45.276926,11.313708,60.876925
247,100.657836,46.097722,100.439036,58.215118,69.641941,65.000000,108.673824,123.004065,52.201533,59.033889,...,28.160256,52.000000,42.941821,68.818602,103.831594,89.286057,45.276926,0.000000,37.121422,50.159745
248,79.259069,10.816654,65.924199,22.472205,37.202150,40.706265,79.397733,98.081599,17.691806,57.070132,...,41.725292,15.297059,23.021729,47.010637,75.663730,55.946403,11.313708,37.121422,0.000000,62.433965


## Preprocesamiento de datos

In [ ]:
# Cargar nodos del archivo parquet
nodos_df = g.get_node_table()

# Cargar matriz de distancias del archivo parquet
matriz_distancias_df = g.get_distance_matrix()

coords = nodos_df[['x', 'y']].values  # para clustering espacial
demandas = nodos_df['demand'].values  # para constraints de capacidad

capacidad_vehiculo = 150 # o el valor específico de este problema según Excel

# Lo volvemos una matriz distancia para que sea mas rapido
matriz_distancias = matriz_distancias_df.values

## Clase del algoritmo de Colonia de Hormigas (ACO)

In [ ]:
class AntColonyVRP:
    def __init__(self, distancia, n_ants=10, n_iter=50, alpha=1.0, beta=5.0, evaporation_rate=0.5, Q=100):
        # Inicialización de parámetros
        self.distancia = distancia                      # Matriz de distancias entre nodos
        self.n = distancia.shape[0]                     # Número de nodos
        self.n_ants = n_ants                            # Número de hormigas por iteración
        self.n_iter = n_iter                            # Número de iteraciones
        self.alpha = alpha                              # Influencia de las feromonas
        self.beta = beta                                # Influencia de la heurística (visibilidad)
        self.evaporation_rate = evaporation_rate        # Tasa de evaporación de feromonas
        self.Q = Q                                      # Factor de depósito de feromonas
        self.feromonas = np.ones((self.n, self.n))      # Matriz inicial de feromonas

        # Visibilidad heurística (1 / distancia) — cuanto más cercana, más atractiva
        with np.errstate(divide='ignore'):
            self.visibilidad = 1 / (distancia + 1e-10)  # Se suma un valor pequeño para evitar división por cero

    def _probabilidades(self, nodo_actual, visitados):
        # Calcula las probabilidades de transición desde el nodo actual a los no visitados
        probs = []
        for j in range(self.n):
            if j in visitados:
                probs.append(0)  # No se puede visitar un nodo ya visitado
            else:
                tau = self.feromonas[nodo_actual][j] ** self.alpha  # Intensidad de feromonas
                eta = self.visibilidad[nodo_actual][j] ** self.beta  # Visibilidad (heurística)
                probs.append(tau * eta)

        suma = sum(probs)
        if suma == 0:
            return np.zeros(self.n)  # Si no hay caminos válidos, se devuelve vector nulo
        return np.array(probs) / suma  # Normaliza las probabilidades

    def _construir_ruta(self):
        # Construye una ruta completa para una hormiga (desde el depósito y de vuelta)
        ruta = [0]  # Se inicia desde el depósito (nodo 0)
        visitados = set(ruta)

        while len(ruta) < self.n:
            actual = ruta[-1]
            probs = self._probabilidades(actual, visitados)
            siguiente = np.random.choice(range(self.n), p=probs)  # Se elige el siguiente nodo según probabilidad
            ruta.append(siguiente)
            visitados.add(siguiente)

        ruta.append(0)  # Vuelve al depósito al final
        return ruta

    def _longitud_ruta(self, ruta):
        # Calcula la longitud total de una ruta (suma de distancias entre pares consecutivos)
        return sum(self.distancia[ruta[i]][ruta[i+1]] for i in range(len(ruta)-1))

    def optimizar(self):
        # Ejecuta el algoritmo ACO durante las iteraciones especificadas
        mejor_ruta = None
        mejor_longitud = float('inf')

        for _ in range(self.n_iter):
            # Cada hormiga construye una ruta
            rutas = [self._construir_ruta() for _ in range(self.n_ants)]
            longitudes = [self._longitud_ruta(r) for r in rutas]

            # Evaporación de feromonas
            self.feromonas *= (1 - self.evaporation_rate)

            # Reforzamiento de feromonas basado en las rutas encontradas
            for ruta, longitud in zip(rutas, longitudes):
                for i in range(len(ruta) - 1):
                    a, b = ruta[i], ruta[i+1]
                    self.feromonas[a][b] += self.Q / longitud

            # Actualización de la mejor ruta encontrada hasta el momento
            min_index = np.argmin(longitudes)
            if longitudes[min_index] < mejor_longitud:
                mejor_longitud = longitudes[min_index]
                mejor_ruta = rutas[min_index]

        return mejor_ruta, mejor_longitud  # Retorna la mejor ruta y su longitud total


## Clusterizacion con ACO greedy

In [ ]:
def greedy_cluster_mejorado(coords, demandas, capacidad, matriz_distancias):
    # Número total de nodos
    n = len(demandas)

    # Conjunto de nodos aún no asignados a ningún cluster (excluyendo el depósito, que es el nodo 0)
    nodos_sin_visitar = set(range(1, n))

    # Lista donde se almacenarán los clusters generados
    clusters = []

    # Se continúa formando clusters hasta que no queden nodos por visitar
    while nodos_sin_visitar:
        cluster = [0]  # Cada cluster comienza en el depósito (nodo 0)
        carga_actual = 0  # Carga acumulada del vehículo
        nodo_actual = 0  # Nodo actual (empieza en el depósito)

        while True:
            # Filtra los nodos candidatos que aún no han sido visitados y que caben en la capacidad restante
            candidatos = [
                nodo for nodo in nodos_sin_visitar
                if demandas[nodo] + carga_actual <= capacidad
            ]

            # Si no hay candidatos válidos, termina la construcción del cluster actual
            if not candidatos:
                break

            # Selecciona el mejor candidato con un balance entre alta demanda y cercanía
            # Se prioriza una alta demanda dividida por la distancia desde el nodo actual
            siguiente = max(
                candidatos,
                key=lambda nodo: (demandas[nodo]) / (matriz_distancias[nodo_actual][nodo] + 1e-6)
                # Se añade 1e-6 para evitar división por cero
            )

            # Añade el nodo al cluster actual
            cluster.append(siguiente)

            # Actualiza la carga total del vehículo
            carga_actual += demandas[siguiente]

            # Elimina el nodo de los que faltan por visitar
            nodos_sin_visitar.remove(siguiente)

            # Actualiza el nodo actual
            nodo_actual = siguiente

        # Añade el cluster completo a la lista de clusters
        clusters.append(cluster)

    # Retorna la lista de clusters generados
    return clusters

In [ ]:
clusters = greedy_cluster_mejorado(coords, demandas, capacidad_vehiculo, matriz_distancias)

In [ ]:
clusters

[[0, 177, 22, 238, 105, 67, 85, 186, 233, 135, 20],
 [0, 21, 122, 182, 171, 52, 127, 108, 50, 139, 188],
 [0, 33, 244, 239, 74, 221, 79, 76, 162, 16, 56],
 [0, 220, 159, 166, 80, 126, 211, 10, 198, 231, 189],
 [0, 7, 157, 6, 178, 205, 55, 46, 124, 245, 117],
 [0, 136, 164, 155, 243, 169, 137, 5, 206, 213, 180],
 [0, 35, 121, 228, 163, 100, 237, 140, 152, 70, 13],
 [0, 60, 208, 227, 174, 2, 146, 94, 34, 101, 151],
 [0, 84, 61, 90, 77, 118, 31, 156, 19, 107, 158],
 [0, 201, 99, 215, 133, 210, 14, 204, 200, 62, 9],
 [0, 111, 4, 130, 96, 43, 115, 3, 97, 113, 45],
 [0, 160, 51, 86, 138, 71, 142, 141, 95, 73, 234],
 [0, 112, 28, 25, 128, 57, 154, 93, 109, 224, 249],
 [0, 12, 236, 153, 246, 134, 1, 192, 131, 172, 30],
 [0, 193, 226, 185, 129, 190, 125, 32, 58, 196, 29],
 [0, 187, 222, 123, 89, 78, 184, 48, 36, 144, 38],
 [0, 242, 176, 82, 75, 69, 24, 202, 91, 23, 65],
 [0, 191, 161, 44, 195, 241, 83, 8, 230, 199, 173],
 [0, 103, 218, 81, 106, 120, 26, 88, 116, 39, 119],
 [0, 47, 181, 248, 11,

## Clase Clarke Wright

In [ ]:
class ClarkWright:
    def __init__(self, distancia, demanda, capacidad_vehiculo):
        # Inicializa el objeto con la matriz de distancias, la demanda por cliente y la capacidad del vehículo
        self.distancia = distancia                    # Matriz de distancias entre nodos
        self.demanda = demanda                        # Lista de demandas por nodo (cliente)
        self.capacidad = capacidad_vehiculo           # Capacidad máxima de cada vehículo
        self.n = len(demanda)                         # Número total de nodos (incluye el depósito)

        # Inicialmente cada cliente tiene su propia ruta: [i]
        self.rutas = [[i] for i in range(1, self.n)]  # Nodo 0 es el depósito, se excluye aquí

        # Lista para guardar los ahorros calculados entre pares de nodos
        self.ahorros = []

    def calcular_ahorros(self):
        # Calcula el "ahorro" que representa unir dos rutas en lugar de enviar dos vehículos separados
        for i in range(1, self.n):            # Nodo 0 es el depósito, así que se empieza en 1
            for j in range(i+1, self.n):
                ahorro = self.distancia[0][i] + self.distancia[0][j] - self.distancia[i][j]
                self.ahorros.append((ahorro, i, j))

        # Ordena los ahorros de mayor a menor para priorizar fusiones más ventajosas
        self.ahorros.sort(reverse=True)

    def fusionar_rutas(self):
        # Intenta fusionar rutas siempre que haya ahorro y no se exceda la capacidad del vehículo
        while self.ahorros:
            _, i, j = self.ahorros.pop(0)  # Toma el par con mayor ahorro

            # Busca las rutas que contienen a i y j (al inicio o al final)
            ruta_i = next((r for r in self.rutas if r[0] == i or r[-1] == i), None)
            ruta_j = next((r for r in self.rutas if r[0] == j or r[-1] == j), None)

            # Si alguna ruta no se encontró o ya son la misma, continúa
            if ruta_i is None or ruta_j is None or ruta_i == ruta_j:
                continue

            # Suma las demandas para verificar que no se exceda la capacidad
            nueva_demanda = sum(self.demanda[n] for n in ruta_i + ruta_j)
            if nueva_demanda > self.capacidad:
                continue

            # Verifica que puedan unirse: el final de una ruta con el inicio de la otra
            if ruta_i[-1] == i and ruta_j[0] == j:
                self.rutas.remove(ruta_i)
                self.rutas.remove(ruta_j)
                self.rutas.append(ruta_i + ruta_j)
            elif ruta_j[-1] == j and ruta_i[0] == i:
                self.rutas.remove(ruta_i)
                self.rutas.remove(ruta_j)
                self.rutas.append(ruta_j + ruta_i)

    def obtener_clusters(self):
        # Retorna las rutas completas, agregando el depósito (nodo 0) al principio y al final
        return [[0] + r + [0] for r in self.rutas]

    def obtener_clusters_sin_deposito(self):
        # Retorna solo los nodos cliente de cada ruta (sin incluir el nodo 0)
        return [r[:] for r in self.rutas]

    def longitud_ruta(self, ruta):
        """
        Calcula la longitud total de una ruta dada (lista de nodos).
        La ruta debe incluir el nodo depósito (0) al inicio y al final.
        """
        longitud = 0
        for i in range(len(ruta) - 1):
            origen = ruta[i]
            destino = ruta[i + 1]
            longitud += self.distancia[origen][destino]
        return longitud

    def longitud_total_rutas(self):
        """
        Calcula la suma de las longitudes de todas las rutas almacenadas en self.rutas,
        considerando que cada ruta debe empezar y terminar en el depósito (0).
        """
        total = 0
        for ruta in self.obtener_clusters():
            total += self.longitud_ruta(ruta)
        return total

    def contar_vehiculos(self, rutas):
        """
        Cuenta cuántos vehículos se están usando (una ruta no vacía representa un vehículo).

        Parámetros:
        - rutas: lista de rutas (cada ruta es una lista de nodos)

        Retorna:
        - int: cantidad de vehículos usados
        """
        return sum(1 for ruta in rutas if ruta)



In [ ]:
cw = ClarkWright(matriz_distancias_df, demandas, capacidad_vehiculo)
cw.calcular_ahorros()
cw.fusionar_rutas()
clusters = cw.obtener_clusters()  # rutas con depósito

print("Longitud total de todas las rutas:", cw.longitud_total_rutas())
for ruta in clusters:
    print(f"Ruta {ruta} tiene longitud {cw.longitud_ruta(ruta)}")

cw.contar_vehiculos(clusters)

Longitud total de todas las rutas: 3998.425901369687
Ruta [0, 229, 92, 114, 145, 168, 183, 203, 42, 247, 149, 0] tiene longitud 256.99055986466965
Ruta [0, 214, 102, 150, 143, 170, 27, 66, 110, 132, 53, 0] tiene longitud 233.73821679983166
Ruta [0, 225, 207, 17, 40, 209, 41, 148, 219, 18, 173, 0] tiene longitud 226.00061536279685
Ruta [0, 68, 15, 104, 87, 217, 54, 167, 64, 72, 212, 0] tiene longitud 204.30461397860427
Ruta [0, 248, 181, 11, 23, 65, 147, 194, 223, 59, 232, 0] tiene longitud 215.3300268811598
Ruta [0, 175, 179, 216, 29, 63, 165, 8, 83, 199, 230, 0] tiene longitud 212.76394464691793
Ruta [0, 26, 197, 235, 38, 37, 98, 144, 36, 240, 119, 0] tiene longitud 196.1788435081827
Ruta [0, 3, 97, 113, 195, 241, 32, 58, 196, 125, 190, 0] tiene longitud 168.20202733441175
Ruta [0, 103, 158, 81, 218, 106, 249, 120, 224, 109, 128, 0] tiene longitud 166.14711691909343
Ruta [0, 45, 1, 192, 30, 172, 131, 134, 246, 153, 44, 0] tiene longitud 159.91647507629764
Ruta [0, 51, 73, 138, 71, 142

25

## Función para aplicar ACO a cada cluster

In [ ]:
def aplicar_aco_a_clusters(clusters, matriz_distancias, n_ants, n_iter, alpha=1, beta=5, evaporation_rate=0.5, Q=100):
    resultados = []          # Lista para guardar los resultados de cada cluster
    distancia_total = 0      # Suma total de las longitudes de las rutas optimizadas

    # Recorre cada cluster de nodos (cada grupo de clientes a ser visitado por un vehículo)
    for cluster in clusters:
        # Extrae la submatriz de distancias entre los nodos del cluster
        sub_dm = matriz_distancias[np.ix_(cluster, cluster)]

        # Crea una instancia del algoritmo ACO con los parámetros dados
        aco = AntColonyVRP(
            sub_dm,
            n_ants=n_ants,
            n_iter=n_iter,
            alpha=alpha,
            beta=beta,
            evaporation_rate=evaporation_rate,
            Q=Q
        )

        # Ejecuta la optimización con ACO para encontrar la mejor ruta dentro del cluster
        mejor_ruta_local, longitud = aco.optimizar()

        # Traduce la mejor ruta de índices locales (del subgrafo) a índices globales (del grafo original)
        mejor_ruta_global = [cluster[i] for i in mejor_ruta_local]

        # Guarda los resultados en la lista
        resultados.append({
            "ruta": mejor_ruta_global,
            "longitud": longitud
        })

        # Acumula la distancia total recorrida
        distancia_total += longitud

    # Número total de vehículos usados (uno por cluster)
    cantidad_vehiculos = len(clusters)

    # Muestra los resultados por consola
    print(f"Cantidad de vehículos: {cantidad_vehiculos}")
    print(f"Distancia total: {distancia_total}")

    # Devuelve la lista de rutas, cantidad de vehículos y distancia total
    return resultados, cantidad_vehiculos, distancia_total


In [ ]:
def aplicar_aco_a_clarke_wright(clusters, matriz_distancias, n_ants=20, n_iter=100, alpha=1, beta=5, evaporation_rate=0.5, Q=100):
    """
    Aplica el algoritmo de colonia de hormigas (ACO) a los clústeres generados por Clarke & Wright.

    Parámetros:
        clusters: lista de listas, donde cada sublista representa los nodos asignados a un vehículo.
        matriz_distancias: matriz de distancias entre nodos.
        n_ants: número de hormigas a usar en ACO.
        n_iter: número de iteraciones del ACO.
        alpha: importancia del rastro de feromonas.
        beta: importancia de la heurística (distancia inversa).
        evaporation_rate: tasa de evaporación de feromonas.
        Q: constante de depósito de feromonas.

    Retorna:
        resultados: lista de rutas optimizadas por ACO para cada clúster.
        cantidad_vehiculos: cantidad de clústeres (vehículos usados).
        distancia_total: suma total de las distancias de las rutas.
    """

    resultados = []         # Lista para guardar los resultados por clúster
    distancia_total = 0     # Suma acumulada de distancias

    # Iterar por cada clúster de nodos (clientes)
    for cluster in clusters:
        # Asegurar que el depósito (nodo 0) esté presente al inicio y final
        if 0 not in cluster:
            nodos_cluster = [0] + cluster + [0]
        else:
            nodos_cluster = cluster

        # Eliminar nodos duplicados si existen
        nodos_cluster = list(dict.fromkeys(nodos_cluster))

        # Crear una submatriz de distancias para el clúster actual
        sub_dm = matriz_distancias[np.ix_(nodos_cluster, nodos_cluster)]

        # Instanciar y ejecutar el algoritmo de colonia de hormigas
        aco = AntColonyVRP(
            sub_dm,
            n_ants=n_ants,
            n_iter=n_iter,
            alpha=alpha,
            beta=beta,
            evaporation_rate=evaporation_rate,
            Q=Q
        )
        mejor_ruta_local, longitud = aco.optimizar()  # Obtener mejor ruta y su longitud

        # Mapear los índices locales del ACO a los índices originales del problema
        idx_map = {i: nodo for i, nodo in enumerate(nodos_cluster)}
        mejor_ruta_global = [idx_map[i] for i in mejor_ruta_local]

        # Guardar los resultados para este clúster
        resultados.append({
            "ruta": mejor_ruta_global,
            "longitud": longitud
        })

        # Acumular la distancia total
        distancia_total += longitud

    # Calcular la cantidad de vehículos utilizados (un vehículo por clúster)
    cantidad_vehiculos = len(clusters)

    # Imprimir resultados generales
    print(f"Cantidad de vehículos: {cantidad_vehiculos}")
    print(f"Distancia total: {distancia_total}")

    return resultados, cantidad_vehiculos, distancia_total


In [ ]:
aplicar_aco_a_clarke_wright(clusters, matriz_distancias,20,100)

Cantidad de vehículos: 25
Distancia total: 3976.4259092542243


([{'ruta': [0, 247, 42, 149, 183, 203, 145, 114, 168, 92, 229, 0],
   'longitud': np.float64(256.1395182776225)},
  {'ruta': [0, 214, 143, 170, 102, 150, 27, 66, 110, 132, 53, 0],
   'longitud': np.float64(230.40347598350945)},
  {'ruta': [0, 173, 18, 219, 209, 148, 41, 17, 40, 207, 225, 0],
   'longitud': np.float64(224.4397448093444)},
  {'ruta': [0, 68, 104, 15, 217, 87, 64, 167, 54, 72, 212, 0],
   'longitud': np.float64(205.74163387158066)},
  {'ruta': [0, 232, 59, 223, 147, 65, 23, 194, 11, 181, 248, 0],
   'longitud': np.float64(217.47686637990284)},
  {'ruta': [0, 199, 230, 83, 8, 165, 29, 63, 179, 216, 175, 0],
   'longitud': np.float64(207.04239743971846)},
  {'ruta': [0, 26, 197, 235, 38, 37, 98, 144, 36, 240, 119, 0],
   'longitud': np.float64(196.1788435081827)},
  {'ruta': [0, 3, 97, 113, 195, 241, 32, 58, 196, 125, 190, 0],
   'longitud': np.float64(168.20202733441175)},
  {'ruta': [0, 128, 109, 224, 120, 249, 106, 218, 81, 158, 103, 0],
   'longitud': np.float64(166.147

In [ ]:
resultados, cantidad_vehiculos, distancia_total = aplicar_aco_a_clusters(
    clusters,
    matriz_distancias,
    n_ants=20,
    n_iter=100,
    alpha=1,
    beta=5,
    evaporation_rate=0.5,
    Q=100
)

Cantidad de vehículos: 25
Distancia total: 3970.918493912194


In [ ]:
cw = ClarkWright(matriz_distancias, demandas, capacidad_vehiculo)
cw.calcular_ahorros()
cw.fusionar_rutas()
clusters = cw.obtener_clusters_sin_deposito()

print("Clusters generados:", clusters)

aplicar_aco_a_clarke_wright(clusters, matriz_distancias, n_ants=20, n_iter=100)

Clusters generados: [[229, 92, 114, 145, 168, 183, 203, 42, 247, 149], [214, 102, 150, 143, 170, 27, 66, 110, 132, 53], [225, 207, 17, 40, 209, 41, 148, 219, 18, 173], [68, 15, 104, 87, 217, 54, 167, 64, 72, 212], [248, 181, 11, 23, 65, 147, 194, 223, 59, 232], [175, 179, 216, 29, 63, 165, 8, 83, 199, 230], [26, 197, 235, 38, 37, 98, 144, 36, 240, 119], [3, 97, 113, 195, 241, 32, 58, 196, 125, 190], [103, 158, 81, 218, 106, 249, 120, 224, 109, 128], [45, 1, 192, 30, 172, 131, 134, 246, 153, 44], [51, 73, 138, 71, 142, 141, 234, 49, 95, 86], [191, 184, 123, 89, 48, 78, 39, 88, 116, 222], [161, 47, 91, 202, 24, 69, 75, 82, 176, 242], [9, 28, 25, 57, 154, 93, 187, 133, 210, 14], [111, 43, 130, 96, 115, 129, 185, 226, 193, 13], [189, 112, 61, 90, 19, 107, 156, 31, 118, 77], [227, 2, 146, 101, 151, 34, 94, 160, 117, 245], [108, 50, 139, 12, 236, 188, 180, 213, 5, 137], [169, 201, 99, 215, 204, 62, 200, 84, 231, 198], [206, 127, 237, 4, 140, 152, 70, 46, 124, 208], [79, 76, 60, 121, 228, 163

([{'ruta': [0, 247, 42, 149, 183, 203, 145, 114, 168, 92, 229, 0],
   'longitud': np.float64(256.1395182776225)},
  {'ruta': [0, 214, 143, 170, 102, 150, 27, 66, 110, 132, 53, 0],
   'longitud': np.float64(230.40347598350945)},
  {'ruta': [0, 173, 18, 219, 209, 41, 148, 17, 40, 207, 225, 0],
   'longitud': np.float64(225.0452960848084)},
  {'ruta': [0, 68, 15, 104, 87, 217, 64, 167, 54, 72, 212, 0],
   'longitud': np.float64(205.0832189109766)},
  {'ruta': [0, 232, 59, 223, 147, 194, 65, 23, 11, 181, 248, 0],
   'longitud': np.float64(213.52656516083715)},
  {'ruta': [0, 175, 216, 179, 63, 29, 165, 8, 83, 230, 199, 0],
   'longitud': np.float64(207.04239743971846)},
  {'ruta': [0, 26, 197, 235, 38, 37, 98, 144, 36, 240, 119, 0],
   'longitud': np.float64(196.1788435081827)},
  {'ruta': [0, 3, 97, 113, 195, 241, 32, 58, 196, 125, 190, 0],
   'longitud': np.float64(168.20202733441175)},
  {'ruta': [0, 128, 109, 224, 120, 249, 106, 218, 81, 158, 103, 0],
   'longitud': np.float64(166.1471

## Visualización de rutas

In [ ]:
a = pd.read_excel("/content/carpeta_descomprimida/problemset/problemset.xlsx")

In [ ]:
lista_de_problemas = a['problem_cluster'].to_list()
lista_de_problemas = list(set(lista_de_problemas))
lista_de_problemas

['big-250n-c150_300-d10_50',
 'medium-50n-c80_120-d10_50',
 'small-10n-c50_70-d10_50',
 'big-250n-c150_300-d15',
 'small-10n-c80_120-d10_50',
 'medium-50n-c150_200-d15',
 'big-250n-c80_120-d10_50',
 'small-10n-c50_60-d10_50',
 'medium-50n-c150_200-d10_50',
 'small-10n-c50-d10_50',
 'small-10n-c30_75-d15']

In [ ]:
#como bf tenia muchos nulos, solo me quede con cw y aco para los benchmarks
benchmark_consulta = a[['problem_cluster','vehicle_capacity','cw best value', 'cw n vehicles', 'aco best value', 'aco n vehicles']]
benchmark_consulta

,problem_cluster,vehicle_capacity,cw best value,cw n vehicles,aco best value,aco n vehicles
0,small-10n-c50_70-d10_50,50,922.314680,6,1090.129161,6
1,small-10n-c50_70-d10_50,55,733.977081,5,945.011879,5
2,small-10n-c50_70-d10_50,60,651.901348,4,693.171486,4
3,small-10n-c50_70-d10_50,65,651.901348,4,708.420574,4
4,small-10n-c50_70-d10_50,70,651.901348,4,769.112343,4
...,...,...,...,...,...,...
73,big-250n-c150_300-d15,222,3196.601990,19,4997.874931,18
74,big-250n-c150_300-d15,240,2892.500183,17,4935.450769,16
75,big-250n-c150_300-d15,258,2799.381625,16,4907.627587,15
76,big-250n-c150_300-d15,276,2702.502351,15,4789.440067,14


In [ ]:
def obtener_datos_grafo(grafo):
    nodos = grafo.get_node_table()

    matriz_distancia = grafo.get_distance_matrix().values
    coords = nodos[['x', 'y']].values  # para clustering espacial
    demandas = nodos['demand'].values  # para constraints de capacidad

    return coords, matriz_distancia, demandas

In [ ]:
def extraer_capacidades(problema):
  capacidades_disponibles = benchmark_consulta[benchmark_consulta['problem_cluster'] == problema]['vehicle_capacity'].to_list()
  return capacidades_disponibles

def resultados(problema):
  folder = "carpeta_descomprimida/problemset/in"

  g = Graph(problema, folder)

  nodos, matriz_distancia = obtener_datos_grafo(g)
  capacidades = extraer_capacidades(problema)

  coords = nodos[['x', 'y']].values  # para clustering espacial
  demandas = nodos['demand'].values  # para constraints de capacidad


In [ ]:
def evaluar_algoritmo_cw(evap_rate, alpha, beta, ants):
  #codigo de evaluacion
  resultados = []
  contador = 0
  error_general_vehiculos = 0
  error_general_distancia = 0
  error_cuadrado_vehiculos = 0
  error_cuadrado_distancia = 0



  for problema in lista_de_problemas:
    folder = "carpeta_descomprimida/problemset/in"

    g = Graph(problema, folder)

    coords, matriz_distancia, demandas = obtener_datos_grafo(g)
    capacidades = extraer_capacidades(problema)

    for capacidad in capacidades:
      cw = ClarkWright(matriz_distancia, demandas, capacidad)
      cw.calcular_ahorros()
      cw.fusionar_rutas()
      clusters = cw.obtener_clusters_sin_deposito()

      _,cantidad_vehiculos,distancia = aplicar_aco_a_clarke_wright(clusters,matriz_distancia,ants,n_iter=15,alpha=alpha,beta=beta,evaporation_rate=evap_rate)

      prueba= benchmark_consulta[(benchmark_consulta['problem_cluster'] == problema) & (benchmark_consulta['vehicle_capacity'] == capacidad)][['aco best value','aco n vehicles']]
      index = int(prueba.index[0])
      problemset_ref = prueba.to_dict()
      aco_best_value = problemset_ref['aco best value'][index]
      aco_n_vehicles = problemset_ref['aco n vehicles'][index]

      error_vehiculos = abs(cantidad_vehiculos - aco_n_vehicles)
      error_distancia = abs(distancia - aco_best_value)

      resultados.append({
          "problema": problema,
          "capacidad": capacidad,
          "vehiculos": cantidad_vehiculos,
          "distancia": distancia,
          "problemset_aco_best_value": aco_best_value,
          "problemset_aco_n_vehicles": aco_n_vehicles,
          "error_abs_vehiculos": error_vehiculos,
          "error_abs_distancia": error_distancia,
          "error_rel_vehiculos": error_vehiculos / aco_n_vehicles if aco_n_vehicles != 0 else float('inf'),
          "error_rel_distancia": error_distancia / aco_best_value if aco_best_value != 0 else float('inf')

      })

      contador += 1
      error_general_vehiculos += error_vehiculos
      error_general_distancia += error_distancia
      error_cuadrado_vehiculos += error_vehiculos ** 2
      error_cuadrado_distancia += error_distancia ** 2
      print(contador)


  error_promedio_vehiculos = error_general_vehiculos / contador
  error_promedio_distancia = error_general_distancia / contador
  error_cuadrad_promedio_vehiculos = error_cuadrado_vehiculos / contador
  error_cuadrad_promedio_distancia = error_cuadrado_distancia / contador

  return resultados, error_promedio_vehiculos, error_promedio_distancia,error_cuadrad_promedio_distancia,error_cuadrad_promedio_vehiculos

In [ ]:
def prueba_bucle():
  for problema in lista_de_problemas:
    print(problema)

    capacidades = extraer_capacidades(problema)
    for i in capacidades:
      print(i)
prueba_bucle()

big-250n-c150_300-d10_50
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
medium-50n-c80_120-d10_50
80
87
94
101
108
115
small-10n-c50_70-d10_50
50
55
60
65
70
big-250n-c150_300-d15
150
168
186
204
222
240
258
276
294
small-10n-c80_120-d10_50
80
100
120
medium-50n-c150_200-d15
150
155
160
165
170
175
180
185
190
195
200
big-250n-c80_120-d10_50
80
87
94
101
108
115
small-10n-c50_60-d10_50
50
52
54
56
58
60
medium-50n-c150_200-d10_50
150
155
160
165
170
175
180
185
190
195
200
small-10n-c50-d10_50
50
small-10n-c30_75-d15
30
45
60
75


In [ ]:
def evaluar_algoritmo_aco(evap_rate, alpha, beta, ants):
  #codigo de evaluacion
  resultados = []
  contador = 0
  error_general_vehiculos = 0
  error_general_distancia = 0
  error_cuadrado_vehiculos = 0
  error_cuadrado_distancia = 0

  for problema in lista_de_problemas:
    folder = "carpeta_descomprimida/problemset/in"

    g = Graph(problema, folder)

    coords, matriz_distancia, demandas = obtener_datos_grafo(g)
    capacidades = extraer_capacidades(problema)

    for capacidad in capacidades:
      clusters = greedy_cluster_mejorado(coords,demandas, capacidad, matriz_distancia)

      _,cantidad_vehiculos,distancia = aplicar_aco_a_clusters(clusters,matriz_distancia,ants,n_iter=15,alpha=alpha,beta=beta,evaporation_rate=evap_rate)

      prueba= benchmark_consulta[(benchmark_consulta['problem_cluster'] == problema) & (benchmark_consulta['vehicle_capacity'] == capacidad)][['aco best value','aco n vehicles']]
      index = int(prueba.index[0])
      problemset_ref = prueba.to_dict()
      aco_best_value = problemset_ref['aco best value'][index]
      aco_n_vehicles = problemset_ref['aco n vehicles'][index]

      error_vehiculos = abs(cantidad_vehiculos - aco_n_vehicles)
      error_distancia = abs(distancia - aco_best_value)

      resultados.append({
          "problema": problema,
          "capacidad": capacidad,
          "vehiculos": cantidad_vehiculos,
          "distancia": distancia,
          "problemset_aco_best_value": aco_best_value,
          "problemset_aco_n_vehicles": aco_n_vehicles,
          "error_abs_vehiculos": error_vehiculos,
          "error_abs_distancia": error_distancia,
          "error_rel_vehiculos": error_vehiculos / aco_n_vehicles if aco_n_vehicles != 0 else float('inf'),
          "error_rel_distancia": error_distancia / aco_best_value if aco_best_value != 0 else float('inf')

      })

      contador += 1
      error_general_vehiculos += error_vehiculos
      error_general_distancia += error_distancia
      error_cuadrado_vehiculos += error_vehiculos ** 2
      error_cuadrado_distancia += error_distancia ** 2


  error_promedio_vehiculos = error_general_vehiculos / contador
  error_promedio_distancia = error_general_distancia / contador
  error_cuadrad_promedio_vehiculos = error_cuadrado_vehiculos / contador
  error_cuadrad_promedio_distancia = error_cuadrado_distancia / contador

  return resultados, error_promedio_vehiculos, error_promedio_distancia,error_cuadrad_promedio_distancia,error_cuadrad_promedio_vehiculos


In [ ]:
resultados, error_promedio_vehiculos, error_promedio_distancia,error_cuadrad_promedio_distancia,error_cuadrad_promedio_vehiculos = evaluar_algoritmo_cw(0.5,1,5,20)

text=f"""
Estadisticas o metricas del algoritmo creado en comparacion con los resultados del problemset:

El error promedio de los vehiculos es: {error_promedio_vehiculos}
El error promedio de la distancia es: {error_promedio_distancia}
El error cuadrado promedio de la distancia es: {error_cuadrad_promedio_distancia}
El error cuadrado promedio de los vehiculos: {error_cuadrad_promedio_vehiculos}
"""

print(text)

Cantidad de vehículos: 52
Distancia total: 5887.198460297889
1
Cantidad de vehículos: 49
Distancia total: 5581.901171979193
2
Cantidad de vehículos: 46
Distancia total: 5275.875467326682
3
Cantidad de vehículos: 43
Distancia total: 5054.7470474387455
4
Cantidad de vehículos: 40
Distancia total: 4807.235225746673
5
Cantidad de vehículos: 38
Distancia total: 4528.691742046704
6
Cantidad de vehículos: 36
Distancia total: 4409.899886195397
7
Cantidad de vehículos: 35
Distancia total: 4256.391357564759
8
Cantidad de vehículos: 33
Distancia total: 4077.578231015589
9
Cantidad de vehículos: 32
Distancia total: 3965.3179207158987
10
Cantidad de vehículos: 31
Distancia total: 3866.43075192434
11
Cantidad de vehículos: 29
Distancia total: 3694.873139578188
12
Cantidad de vehículos: 28
Distancia total: 3638.492947208487
13
Cantidad de vehículos: 28
Distancia total: 3523.2195994755834
14
Cantidad de vehículos: 26
Distancia total: 3415.628222793878
15
Cantidad de vehículos: 25
Distancia total: 3348

In [ ]:
results = pd.DataFrame(resultados)
results

,problema,capacidad,vehiculos,distancia,problemset_aco_best_value,problemset_aco_n_vehicles,error_abs_vehiculos,error_abs_distancia,error_rel_vehiculos,error_rel_distancia
0,big-250n-c150_300-d10_50,150,52,5887.198460,8386.293980,51,1,2499.095520,0.019608,0.297998
1,big-250n-c150_300-d10_50,160,49,5581.901172,8105.215521,47,2,2523.314349,0.042553,0.311320
2,big-250n-c150_300-d10_50,170,46,5275.875467,7796.670406,44,2,2520.794939,0.045455,0.323317
3,big-250n-c150_300-d10_50,180,43,5054.747047,7503.070106,42,1,2448.323059,0.023810,0.326310
4,big-250n-c150_300-d10_50,190,40,4807.235226,7208.194143,40,0,2400.958917,0.000000,0.333087
...,...,...,...,...,...,...,...,...,...,...
73,small-10n-c50-d10_50,50,7,597.575580,597.575580,7,0,0.000000,0.000000,0.000000
74,small-10n-c30_75-d15,30,5,672.154040,811.343549,5,0,139.189508,0.000000,0.171554
75,small-10n-c30_75-d15,45,3,543.033819,550.163849,3,0,7.130030,0.000000,0.012960
76,small-10n-c30_75-d15,60,3,469.422385,485.993131,3,0,16.570746,0.000000,0.034097


In [ ]:
results.to_csv('resultados_benchmark.csv', index=False)

In [ ]:
dropdown = widgets.Dropdown(
    options=lista_de_problemas,
    value=lista_de_problemas[0],
    description='Problema:',
)

boton = widgets.Button(description='Mostrar')

# Mostrar los widgets en la interfaz
display(dropdown, boton)

def dibujar_rutas(coords, resultados, depot=0):

    g = nx.DiGraph()

    # Crear diccionario de posiciones: {nodo_id: (x, y)}
    pos = {i: tuple(coord) for i, coord in enumerate(coords)}
    g.add_nodes_from(pos)

    # Agregar aristas basadas en las rutas de cada vehiculo
    for vid, resultado in enumerate(resultados):
        ruta = resultado["ruta"]
        for u, v in zip(ruta[:-1], ruta[1:]):
            g.add_edge(u, v, vehiculo=vid)

    plt.figure(figsize=(8, 8))

    nx.draw_networkx_nodes(
        g, pos,
        nodelist=[depot],
        node_size=300,
        node_color='red',
        label='Deposito'
    )

    nx.draw_networkx_nodes(
        g, pos,
        nodelist=[n for n in pos if n != depot],
        node_size=100,
        node_color='white',
        edgecolors='black'
    )

    nx.draw_networkx_labels(g, pos, font_size=9)
    colors = plt.cm.get_cmap('tab10', len(resultados))
    n = len(resultados)

    # Dibujar aristas para cada vehículo
    for vid in range(n):
        # Filtrar aristas que pertenecen al vehiculo
        eds = [(u, v) for u, v, d in g.edges(data=True) if d['vehiculo'] == vid]

        # Calcular radio de curvatura para evitar superposicion
        rad = (vid/(n-1) - 0.5) * 0.6 if n > 1 else 0

        # Dibujar aristas del vehiculo
        nx.draw_networkx_edges(
            g, pos,
            edgelist=eds,
            edge_color=[colors(vid)],
            width=2,
            arrowstyle='-|>',
            arrowsize=12,
            connectionstyle=f'arc3,rad={rad}',
            label=f"Vehículo {vid+1}",
            alpha=0.8
        )

    plt.legend(loc='upper right')
    plt.axis('off')
    plt.title("Rutas encontradas")
    plt.show()

def on_button_clicked(b):

    # Limpiar salida anterior y redisplayar interfaz
    clear_output(wait=True)
    display(dropdown, boton)

    problema = dropdown.value

    carpeta = "/content/carpeta_descomprimida/problemset/in"

    # Crear instancia del problema usando clase Graph personalizada
    g_problem = Graph(problema, carpeta)

    nodos_df = g_problem.get_node_table()
    dm_df = g_problem.get_distance_matrix()

    # Convertir datos a arrays numpy para procesamiento
    coords = nodos_df[['x', 'y']].values
    demandas = nodos_df['demand'].values

    # Parámetros del problema
    capacidad_vehiculo = 55
    matriz_distancias = dm_df.values

    print(f"Problema: {problema}")
    display(nodos_df)
    display(dm_df)

    # Fase 1: Clustering greedy para agrupar nodos por capacidad
    clusters = greedy_cluster_mejorado(
        coords,               # Coordenadas de nodos
        demandas,            # Demandas de nodos
        capacidad_vehiculo,  # Capacidad máxima
        matriz_distancias    # Matriz de distancias
    )

    # Optimización ACO para cada cluster
    resultados, nveh, distot = aplicar_aco_a_clusters(
        clusters,
        matriz_distancias,
        n_ants=20,
        n_iter=100,
        alpha=1,            # Factor de influencia de feromona
        beta=5,             # Factor de influencia de distancia
        evaporation_rate=0.5, # Tasa de evaporación de feromona
        Q=100               # Factor de depósito de feromona
    )

    print(f"\nVehiculos usados: {nveh}")
    print(f"\nDistancia total: {distot:.2f}")

    # Visualizar rutas encontradas
    dibujar_rutas(coords, resultados)

boton.on_click(on_button_clicked)

Dropdown(description='Problema:', options=('big-250n-c150_300-d10_50', 'medium-50n-c80_120-d10_50', 'small-10n…

Button(description='Mostrar', style=ButtonStyle())

## Evaluación y validación de resultados

In [ ]:
def extraer_clusters_sin_deposito(rutas):
    return [list(r) for r in rutas]

def calcular_distancia_total(matriz_distancias, rutas, agregar_deposito=True):
    total = 0
    for ruta in rutas:
        if agregar_deposito:
            ruta = [0] + list(ruta) + [0]
        for i in range(len(ruta) - 1):
            total += matriz_distancias[ruta[i], ruta[i+1]]  # Usa coma para índice 2D en numpy
    return total

# Cargar archivo con todos los problemas
df_problemas = pd.read_excel("carpeta_descomprimida/problemset/problemset.xlsx")  # Asegúrate de tener este archivo

df_problemas["nodes"] = df_problemas["nodes"].str.replace(
    "problemset/in/", "carpeta_descomprimida/problemset/in/", regex=False
)
df_problemas["distance_matrix"] = df_problemas["distance_matrix"].str.replace(
    "problemset/in/", "carpeta_descomprimida/problemset/in/", regex=False
)

resultados = []

for _, fila in df_problemas.iterrows():
    try:
        # Leer información del problema
        problem_name = fila['problem_cluster']
        nodes_path = fila['nodes']
        dm_path = fila['distance_matrix']
        capacidad_vehiculo = fila['vehicle_capacity']

        # Cargar datos
        nodos = pd.read_parquet(nodes_path)
        matriz_distancias = np.array(pd.read_parquet(dm_path))
        demandas = nodos['demand'].to_numpy()
        print(f"Procesando problema: {problem_name}")
        print(f"Matriz distancias: shape={matriz_distancias.shape}")

        # Verificar que la matriz sea cuadrada
        if matriz_distancias.shape[0] != matriz_distancias.shape[1]:
            raise ValueError("La matriz de distancias debe ser cuadrada")

        # --- Clarke-Wright ---
        cw = ClarkWright(matriz_distancias, demandas, capacidad_vehiculo)
        cw.calcular_ahorros()
        cw.fusionar_rutas()
        clusters_cw = cw.obtener_clusters_sin_deposito()

        # Validar que todos los nodos estén dentro del rango
        max_nodo = matriz_distancias.shape[0] - 1
        for ruta in clusters_cw:
            for nodo in ruta:
                if nodo > max_nodo or nodo < 0:
                    raise IndexError(f"Índice de nodo fuera de rango en clusters CW: {nodo}")

        distancia_cw = calcular_distancia_total(matriz_distancias, clusters_cw)
        n_vehiculos_cw = len(clusters_cw)

        # --- ACO sobre clusters CW (cw + aco) ---
        aco_cw = aplicar_aco_a_clarke_wright(
            matriz_distancias, clusters_cw,
            n_ants=10, n_iter=50, alpha=1.0, beta=5.0,
            evaporation_rate=0.5, Q=100
        )
        mejor_ruta_cw_aco, mejor_valor_cw_aco = aco_cw.ejecutar()
        n_vehiculos_cw_aco = len(mejor_ruta_cw_aco)

        # --- ACO Greedy desde cero ---
        aco_greedy = aplicar_aco_a_clusters(
            matriz_distancias, None, n_iter=100,
            alfa=1, beta=2, rho=0.5,
            demandas=demandas, capacidad=capacidad_vehiculo
        )
        mejor_ruta_greedy, mejor_valor_greedy = aco_greedy.ejecutar()
        n_vehiculos_greedy = len(mejor_ruta_greedy)

        resultados.append({
            "problem_cluster": problem_name,

            "cw best route": clusters_cw,
            "cw best value": distancia_cw,
            "cw n vehicles": n_vehiculos_cw,

            "cw + aco best route": mejor_ruta_cw_aco,
            "cw + aco best value": mejor_valor_cw_aco,
            "cw + aco n vehicles": n_vehiculos_cw_aco,

            "aco greedy best route": mejor_ruta_greedy,
            "aco greedy best value": mejor_valor_greedy,
            "aco greedy n vehicles": n_vehiculos_greedy
        })

    except Exception as e:
        print(f"Error en {fila['problem_cluster']}: {e}")

# Guardar en DataFrame
df_resultados = pd.DataFrame(resultados)

# Exportar a Excel o CSV si lo deseas
df_resultados.to_excel("resultados_heuristicas.xlsx", index=False)


Procesando problema: small-10n-c50_70-d10_50
Matriz distancias: shape=(10, 10)
Error en small-10n-c50_70-d10_50: list indices must be integers or slices, not tuple
Procesando problema: small-10n-c50_70-d10_50
Matriz distancias: shape=(10, 10)
Error en small-10n-c50_70-d10_50: list indices must be integers or slices, not tuple
Procesando problema: small-10n-c50_70-d10_50
Matriz distancias: shape=(10, 10)
Error en small-10n-c50_70-d10_50: list indices must be integers or slices, not tuple
Procesando problema: small-10n-c50_70-d10_50
Matriz distancias: shape=(10, 10)
Error en small-10n-c50_70-d10_50: list indices must be integers or slices, not tuple
Procesando problema: small-10n-c50_70-d10_50
Matriz distancias: shape=(10, 10)
Error en small-10n-c50_70-d10_50: list indices must be integers or slices, not tuple
Procesando problema: small-10n-c50_60-d10_50
Matriz distancias: shape=(10, 10)
Error en small-10n-c50_60-d10_50: list indices must be integers or slices, not tuple
Procesando probl

In [ ]:
import os

ruta = 'carpeta_descomprimida/problemset/in/small-10n-c50_70-d10_50_nodes.parquet'
print("Existe:", os.path.exists(ruta))

Existe: True
